In [19]:
import pandas as pd
import numpy as np 
import seaborn as sns
from IPython.display import display


#load 
df=pd.read_csv("C:\\Users\\bessa\\Documents\\ooredoo intern\\yacinedata.xlsx - Feuil1.csv")
df.head(5)

,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
0,1134229,AppSpot,WebApps,7.691974e+10,436943480.0,2025-07
1,10131,SIP,Communications,4.441881e+10,525385400.0,2025-07
2,759,RedFlagDeals,SocialNet,9.007530e+07,233200.0,2025-07
3,6,NowPlayer,Video,6.485430e+06,1120.0,2025-07
4,33757,YandexDisk,CloudStorage,5.071030e+12,14042880.0,2025-07


In [20]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   COUNT_DISTINCT_of_subscriber_id  2199 non-null   int64  
 1   application_name                 2137 non-null   object 
 2   category_name                    2193 non-null   object 
 3   SUM_of_total_volume_bytes        2193 non-null   float64
 4   SUM_of_duration                  2193 non-null   float64
 5   event_date_id                    2199 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 103.2+ KB


(2199, 6)

In [21]:
print(df.isna().sum())
print("Duplicated values\n",df.duplicated().sum())
print("\nnegative duration? :",df[df["SUM_of_duration"]<0])


COUNT_DISTINCT_of_subscriber_id     0
application_name                   62
category_name                       6
SUM_of_total_volume_bytes           6
SUM_of_duration                     6
event_date_id                       0
dtype: int64
Duplicated values
 0

negative duration? : Empty DataFrame
Columns: [COUNT_DISTINCT_of_subscriber_id, application_name, category_name, SUM_of_total_volume_bytes, SUM_of_duration, event_date_id]
Index: []


In [22]:
print(df["category_name"].unique())
missing_category_apps = df[df['category_name'].isna()]['application_name'].unique()
print("\nApplication names without category names:", missing_category_apps)

['WebApps' 'Communications' 'SocialNet' 'Video' 'CloudStorage' 'Telco'
 'Finance' 'Remote' 'Technology' 'Ads' 'eCommerce' 'Update' 'IPCamera'
 'Games' 'Web' 'Net' 'Misc' 'Productivity' 'AppleServices' 'Media'
 'Enterprise' 'Music' 'LocationServices' 'Tracker' 'News' 'DeviceServices'
 'CloudServices' 'FileSharing' 'P2P' 'ApplicationStore' 'VPN' 'Proxy'
 'non-established' 'Images' 'Gov' 'Security' 'Sync' 'Education' 'Religion'
 'CDN' 'Transportation' 'Attack' 'Mail' 'Lifestyle' 'YahooServices'
 'Content' 'Travel' 'Sports' nan 'Backup' 'Weather' 'FoodDrinks'
 'JobSearch' 'GoogleServices' 'SearchEngine' 'MDM' 'RPC' 'HomeDecoration'
 'Automotive' 'File' 'RealEstate' 'Banking' 'unclassified' 'DlMgr']

Application names without category names: [nan 'GooglePlayStore' 'empty' 'unclassified' 'NTP' 'TechnologySites']


1)dropping the row that has an application name and category_name as nan<br>
2)
filling the category_name of the missing applications

GooglePlayStore → ApplicationStore
NTP → Net
TechnologySites →Technology
unclassified → Unknown


In [23]:

df = df.dropna(subset=['application_name', 'category_name'], how='all')
print(df.isna().sum())


#mapping
app_category_mapping={
    'GooglePlayStore': 'ApplicationStore',
    'NTP': 'Net',
    'TechnologySites': 'Technology',
    'unclassified': 'Unknown',
    'empty': 'Unknown'   }


df["category_name"]=df.apply(
    lambda row: app_category_mapping.get(row["application_name"],row["category_name"]),axis=1
)

COUNT_DISTINCT_of_subscriber_id     0
application_name                   61
category_name                       5
SUM_of_total_volume_bytes           6
SUM_of_duration                     5
event_date_id                       0
dtype: int64


In [24]:
missing_volume=df[df["SUM_of_total_volume_bytes"].isna()]
display(missing_volume.sort_values('category_name'))


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
724,1,GooglePlayStore,ApplicationStore,NaN,NaN,2025-07
1640,1,NTP,Net,NaN,NaN,2025-07
1899,1,TechnologySites,Technology,NaN,NaN,2025-07
983,1,empty,Unknown,NaN,NaN,2025-07
1107,1,unclassified,Unknown,NaN,NaN,2025-07
1365,0,NaN,unclassified,NaN,568.0,2025-07


since the COUNT_DISTINCT_of_subscriber_id  we see them as all one and only one 0 we drop all of them rows that have subscribers<=1. 

In [25]:
df=df[~((df["COUNT_DISTINCT_of_subscriber_id"].isna()| df['SUM_of_duration'].isna())&
        (df['COUNT_DISTINCT_of_subscriber_id'] <= 1))]

In [26]:
missing_volume1=df[df["SUM_of_total_volume_bytes"].isna()]
display(missing_volume1.sort_values('category_name'))


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
1365,0,NaN,unclassified,NaN,568.0,2025-07


In [27]:
df.drop(1365, inplace=True)


In [28]:
df.isna().sum()

COUNT_DISTINCT_of_subscriber_id     0
application_name                   60
category_name                       0
SUM_of_total_volume_bytes           0
SUM_of_duration                     0
event_date_id                       0
dtype: int64

In [29]:
df['application_name'] = df['application_name'].fillna('Unknown')
bad_apps = ['empty', 'unclassified', 'Unknown']

rows_with_bad_apps = df[df['application_name'].isin(bad_apps)]
#print(rows_with_bad_apps)

bad_categories = ['Unknown', 'unclassified']

# Drop rows where app and category is in the bad list
df = df[~(df['application_name'].isin(bad_apps) & df['category_name'].isin(bad_categories))]

print(df.isna().sum())

COUNT_DISTINCT_of_subscriber_id    0
application_name                   0
category_name                      0
SUM_of_total_volume_bytes          0
SUM_of_duration                    0
event_date_id                      0
dtype: int64


In [30]:
# Count distinct users with unknown apps
unknown_users = df[df["application_name"].str.lower() == "unknown"]["COUNT_DISTINCT_of_subscriber_id"].nunique()

# Count total distinct users
total_users = df["COUNT_DISTINCT_of_subscriber_id"].nunique()

print(f"Users with unknown apps: {unknown_users}")
print(f"Total users: {total_users}")
print(f"Percentage: {unknown_users / total_users * 100:.2f}%")


Users with unknown apps: 60
Total users: 1595
Percentage: 3.76%


Turning app name and category name into lowecase.


In [31]:
df['category_name'] = df['category_name'].str.lower()
df['application_name'] = df['application_name'].str.lower()


In [32]:
apps_low_users=df[df["COUNT_DISTINCT_of_subscriber_id"]<=5]
apps_low_users


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
17,3,symphony,communications,35132368.0,2260.0,2025-07
18,1,livenow,video,18128.0,120.0,2025-07
24,1,gogoanimes,video,61906.0,180.0,2025-07
36,1,buzznet,socialnet,101014.0,80.0,2025-07
40,2,duonao,video,42190.0,120.0,2025-07
...,...,...,...,...,...,...
2159,3,primewire,video,86558.0,220.0,2025-07
2160,3,quizilla,socialnet,46956.0,200.0,2025-07
2167,1,youtubeinmp3,video,28890.0,40.0,2025-07
2186,4,axis,telco,1655252.0,540.0,2025-07


In [33]:
grouped = apps_low_users.groupby('category_name')[['COUNT_DISTINCT_of_subscriber_id',
                                               'SUM_of_total_volume_bytes',
                                               'SUM_of_duration']].sum()

grouped

,COUNT_DISTINCT_of_subscriber_id,SUM_of_total_volume_bytes,SUM_of_duration
category_name,,,
communications,131,4.448875e+08,435480.0
socialnet,175,1.220198e+09,155400.0
telco,113,2.862976e+09,45500.0
video,245,3.765758e+09,151580.0


So here we have 131 users of category communications that are placed under diffrent apps names and their users are less | = to 5 
so i will group the apps that have users <=5 into one app name of the same category
like </br>
COUNT_DISTINCT_of_subscriber_id <=5 </br>
app1 app2 app3 </br>
cat1 cat2 cat1 </br>
here i will group app1 and app3 into cat1

In [34]:
# Filter apps with <=5 users AND category SocialNet
small_socialnet_apps = df[(df['COUNT_DISTINCT_of_subscriber_id'] <= 5) & 
                          (df['category_name'] == 'socialnet')]
small_socialnet_apps


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
36,1,buzznet,socialnet,101014.0,80.0,2025-07
42,3,wayn,socialnet,71058.0,740.0,2025-07
94,1,hospitalityclub,socialnet,16620.0,20.0,2025-07
95,1,zenly,socialnet,15626.0,80.0,2025-07
113,2,kaioo,socialnet,5147790.0,4940.0,2025-07
...,...,...,...,...,...,...
2070,1,dayre,socialnet,14746.0,20.0,2025-07
2093,2,2ch,socialnet,18201412.0,620.0,2025-07
2111,1,mixxer,socialnet,2469852.0,740.0,2025-07
2127,4,mosheng,socialnet,1026628.0,1440.0,2025-07


Now combine all of these 78 row of socialnet category into one app name which will be socialnet_apps under category socialnet

In [35]:
summed = small_socialnet_apps[['COUNT_DISTINCT_of_subscriber_id', 
                                'SUM_of_total_volume_bytes', 
                                'SUM_of_duration']].sum()

In [36]:
# 1️⃣ Drop the old tiny apps
df = df.drop(small_socialnet_apps.index)

# 2️⃣ Create the merged row as a DataFrame
merged_row_df = pd.DataFrame([{
    'COUNT_DISTINCT_of_subscriber_id': summed['COUNT_DISTINCT_of_subscriber_id'],
    'application_name': 'socialnet_apps',
    'category_name': 'socialnet',
    'SUM_of_total_volume_bytes': summed['SUM_of_total_volume_bytes'],
    'SUM_of_duration': summed['SUM_of_duration'],
    'event_date_id': '2025-07'
}])

In [37]:
df = pd.concat([df, merged_row_df], ignore_index=True)


In [38]:
display(df[df['application_name'] == 'socialnet_apps'])


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
2112,175.0,socialnet_apps,socialnet,1.220198e+09,155400.0,2025-07


In [41]:

threshold = 5

# Get all categories that have small apps(expect socialnet   i already did it )
categories = df[df['COUNT_DISTINCT_of_subscriber_id'] <= threshold]['category_name'].unique()

for cat in categories:
    # Filter tiny apps in this category
    small_apps = df[(df['COUNT_DISTINCT_of_subscriber_id'] <= threshold) & 
                    (df['category_name'] == cat)]
    
    # Sum numeric columns
    summed = small_apps[['COUNT_DISTINCT_of_subscriber_id', 
                         'SUM_of_total_volume_bytes', 
                         'SUM_of_duration']].sum()
    
    # Create new merged row
    merged_row_df = pd.DataFrame([{
        'COUNT_DISTINCT_of_subscriber_id': summed['COUNT_DISTINCT_of_subscriber_id'],
        'application_name': f'{cat}_apps',
        'category_name': cat,
        'SUM_of_total_volume_bytes': summed['SUM_of_total_volume_bytes'],
        'SUM_of_duration': summed['SUM_of_duration'],
        'event_date_id': '2025-07'
    }])
    
    # Drop the old small apps
    df = df.drop(small_apps.index)
    
    # Add the new merged app
    df = pd.concat([df, merged_row_df], ignore_index=True)

# ✅ Check result
display(df[df['application_name'].str.endswith('_apps')])


,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
1891,175.0,socialnet_apps,socialnet,1.220198e+09,155400.0,2025-07
1892,131.0,communications_apps,communications,4.448875e+08,435480.0,2025-07
1893,245.0,video_apps,video,3.765758e+09,151580.0,2025-07
1894,113.0,telco_apps,telco,2.862976e+09,45500.0,2025-07


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   COUNT_DISTINCT_of_subscriber_id  1895 non-null   float64
 1   application_name                 1895 non-null   object 
 2   category_name                    1895 non-null   object 
 3   SUM_of_total_volume_bytes        1895 non-null   float64
 4   SUM_of_duration                  1895 non-null   float64
 5   event_date_id                    1895 non-null   object 
dtypes: float64(3), object(3)
memory usage: 89.0+ KB


In [42]:
unknown_apps_grouped = (
    df[df["application_name"].str.lower() == "unknown"]
    .groupby("category_name")
    .sum()
    .reset_index()
)

unknown_apps_grouped

,category_name,COUNT_DISTINCT_of_subscriber_id,application_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
0,ads,583689.0,unknown,5.645737e+11,1.093455e+08,2025-07
1,appleservices,4888.0,unknown,3.878160e+08,8.659100e+05,2025-07
2,applicationstore,21200.0,unknown,1.169495e+12,1.102977e+07,2025-07
3,attack,33293.0,unknown,4.298915e+10,1.600575e+07,2025-07
4,automotive,28064.0,unknown,1.399108e+11,1.689876e+07,2025-07
5,backup,27469.0,unknown,1.020452e+11,3.266393e+07,2025-07
6,banking,46233.0,unknown,1.225103e+11,1.846111e+07,2025-07
7,cdn,66416.0,unknown,2.853587e+12,2.728548e+07,2025-07
8,cloudservices,32289.0,unknown,1.032114e+11,3.120609e+07,2025-07
9,cloudstorage,23311.0,unknown,1.588843e+12,4.592011e+07,2025-07


Now merging the apps that are unkown into other apps of the same category would make the dataset much cleaner </br>
like 0 ads 583689.0 </br> i can merge this uknown app into another app of the same category i.e ads

In [43]:
df[df["category_name"]=="ads"]
#i can merge the 1st row of the unkown app into one of these since they all have the same category

,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
14,19343.0,adzerk,ads,1.105104e+09,5.761220e+06,2025-07
19,1700149.0,moloco,ads,8.868837e+12,7.067883e+08,2025-07
34,16858.0,swrve,ads,9.580623e+09,4.070450e+06,2025-07
51,4484773.0,adjust,ads,1.112717e+12,4.381658e+09,2025-07
54,708392.0,sitescout,ads,2.120744e+10,1.396311e+08,2025-07
...,...,...,...,...,...,...
1813,238660.0,dtscout,ads,1.550056e+10,7.407355e+07,2025-07
1831,54488.0,kika-backend,ads,1.774682e+10,1.065267e+08,2025-07
1836,743856.0,adkernel,ads,4.792424e+10,7.004043e+07,2025-07
1851,430161.0,dataxu,ads,7.015654e+09,7.774934e+07,2025-07


In [44]:
# 1. Pick the unknown apps of category 'ads' and sum their values
unknown_ads = df[(df['application_name'].str.lower() == 'unknown') & (df['category_name'] == 'ads')].sum(numeric_only=True)

# 2. Pick the row of the real app to merge into (adzerk)
ads_row_idx = df[(df['category_name'] == 'ads') & (df['application_name'] == 'adzerk')].index[0]

# 3. Add the counts from unknown apps to adzerk
df.at[ads_row_idx, 'COUNT_DISTINCT_of_subscriber_id'] += unknown_ads['COUNT_DISTINCT_of_subscriber_id']
df.at[ads_row_idx, 'SUM_of_total_volume_bytes'] += unknown_ads['SUM_of_total_volume_bytes']
df.at[ads_row_idx, 'SUM_of_duration'] += unknown_ads['SUM_of_duration']

# 4. Drop the original unknown rows for ads
df = df[~((df['application_name'].str.lower() == 'unknown') & (df['category_name'] == 'ads'))]

# 5. Check the updated row
df.loc[ads_row_idx]


COUNT_DISTINCT_of_subscriber_id          603032.0
application_name                           adzerk
category_name                                 ads
SUM_of_total_volume_bytes          565678798941.0
SUM_of_duration                       115106710.0
event_date_id                             2025-07
Name: 14, dtype: object

Now we can loop instead of doing one by one

In [45]:
unknown_apps_categories = df[df['application_name'].str.lower() == 'unknown']['category_name'].unique()

for cat in unknown_apps_categories:
    unknown_sum = df[(df['application_name'].str.lower() == 'unknown') & (df['category_name'] == cat)].sum(numeric_only=True)
    
    # Get real apps in the same category
    real_apps = df[(df['category_name'] == cat) & (df['application_name'].str.lower() != 'unknown')]
    
    if len(real_apps) == 0:
        # Skip if no real app exists to merge into
        continue
    
    # Merge into the first real app
    real_app_idx = real_apps.index[0]
    df.at[real_app_idx, 'COUNT_DISTINCT_of_subscriber_id'] += unknown_sum['COUNT_DISTINCT_of_subscriber_id']
    df.at[real_app_idx, 'SUM_of_total_volume_bytes'] += unknown_sum['SUM_of_total_volume_bytes']
    df.at[real_app_idx, 'SUM_of_duration'] += unknown_sum['SUM_of_duration']
    
    # Drop the unknown apps in this category
    df = df[~((df['application_name'].str.lower() == 'unknown') & (df['category_name'] == cat))]


In [46]:
df.shape

(1837, 6)

In [47]:
#verify if there still unknown apps
df[df["application_name"]=="unknown"]

,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id
35,3529.0,unknown,misc,3.183588e+09,896820.0,2025-07
131,1730.0,unknown,sync,4.658605e+10,2360600.0,2025-07


merge sync into one of the apps that have category name = to CloudServices or CloudStorage and </br>
and leave the misc unknown app 

In [48]:
# Merge 'sync' unknown apps into 'CloudServices'
# Pick the unknown 'sync' row which is into row 131
unknown_sync = df[(df['application_name'].str.lower() == 'unknown') & (df['category_name'].str.lower() == 'sync')].iloc[0]

# Pick a real app in CloudServices to merge into 
cloud_idx = df[(df['category_name'].str.lower() == 'cloudservices') & (df['application_name'].str.lower() != 'unknown')].index[0]

# Add counts and sums
df.at[cloud_idx, 'COUNT_DISTINCT_of_subscriber_id'] += unknown_sync['COUNT_DISTINCT_of_subscriber_id']
df.at[cloud_idx, 'SUM_of_total_volume_bytes'] += unknown_sync['SUM_of_total_volume_bytes']
df.at[cloud_idx, 'SUM_of_duration'] += unknown_sync['SUM_of_duration']

# Drop the original unknown 'sync' row
df = df[~((df['application_name'].str.lower() == 'unknown') & (df['category_name'].str.lower() == 'sync'))]


In [49]:

idx = 35
df.at[idx, 'application_name'] = 'misc_app'
df.loc[idx]


COUNT_DISTINCT_of_subscriber_id          3529.0
application_name                       misc_app
category_name                              misc
SUM_of_total_volume_bytes          3183587599.0
SUM_of_duration                        896820.0
event_date_id                           2025-07
Name: 35, dtype: object

In [50]:
df[df["application_name"]=="unknown"]

,COUNT_DISTINCT_of_subscriber_id,application_name,category_name,SUM_of_total_volume_bytes,SUM_of_duration,event_date_id


In [51]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1836 entries, 0 to 1894
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   COUNT_DISTINCT_of_subscriber_id  1836 non-null   float64
 1   application_name                 1836 non-null   object 
 2   category_name                    1836 non-null   object 
 3   SUM_of_total_volume_bytes        1836 non-null   float64
 4   SUM_of_duration                  1836 non-null   float64
 5   event_date_id                    1836 non-null   object 
dtypes: float64(3), object(3)
memory usage: 164.9+ KB


In [52]:
# 1. Create behavioral intensity features (these help predict age!)
df['avg_volume_per_user'] = df['SUM_of_total_volume_bytes'] / df['COUNT_DISTINCT_of_subscriber_id']
df['avg_duration_per_user'] = df['SUM_of_duration'] / df['COUNT_DISTINCT_of_subscriber_id']
df['bytes_per_second'] = df['SUM_of_total_volume_bytes'] / (df['SUM_of_duration'] + 1)

In [53]:
# 2. Log transform skewed data for clustering
df['log_volume'] = np.log1p(df['SUM_of_total_volume_bytes'])
df['log_users'] = np.log1p(df['COUNT_DISTINCT_of_subscriber_id'])
df['log_duration'] = np.log1p(df['SUM_of_duration'])

In [54]:
# 3. Create category usage matrix (pivot table - key for age prediction!)
category_pivot = df.groupby('category_name').agg({
    'COUNT_DISTINCT_of_subscriber_id': 'sum',
    'SUM_of_total_volume_bytes': 'sum'
}).reset_index()

In [56]:
print("New features created for clustering:")
display(df[['application_name', 'avg_volume_per_user', 'avg_duration_per_user', 'log_volume']].head())

New features created for clustering:


,application_name,avg_volume_per_user,avg_duration_per_user,log_volume
0,appspot,7.441778e+04,383.230271,25.169276
1,sip,3.813641e+06,43735.151717,24.549171
2,redflagdeals,2.980013e+06,200.128170,24.206064
3,nowplayer,1.612846e+07,450.578113,27.065409
4,yandexdisk,1.167007e+08,1050.728780,29.527122


In [37]:
# Convert COUNT_DISTINCT_of_subscriber_id to int
df["COUNT_DISTINCT_of_subscriber_id"] = df["COUNT_DISTINCT_of_subscriber_id"].astype(int)

# Convert event_date_id to datetime (format YYYY-MM)
df["event_date_id"] = pd.to_datetime(df["event_date_id"], format="%Y-%m")

# Vérifier les nouveaux types
print(df.dtypes)


COUNT_DISTINCT_of_subscriber_id             int64
application_name                           object
category_name                              object
SUM_of_total_volume_bytes                 float64
SUM_of_duration                           float64
event_date_id                      datetime64[ns]
avg_volume_per_user                       float64
avg_duration_per_user                     float64
bytes_per_second                          float64
log_volume                                float64
log_users                                 float64
log_duration                              float64
dtype: object


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1836 entries, 0 to 1894
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   COUNT_DISTINCT_of_subscriber_id  1836 non-null   int64         
 1   application_name                 1836 non-null   object        
 2   category_name                    1836 non-null   object        
 3   SUM_of_total_volume_bytes        1836 non-null   float64       
 4   SUM_of_duration                  1836 non-null   float64       
 5   event_date_id                    1836 non-null   datetime64[ns]
 6   avg_volume_per_user              1836 non-null   float64       
 7   avg_duration_per_user            1836 non-null   float64       
 8   bytes_per_second                 1836 non-null   float64       
 9   log_volume                       1836 non-null   float64       
 10  log_users                        1836 non-null   float64       
 

In [39]:
df.to_csv('data_cleaned_final.csv', index=False)
